### Overview
To gain in explainability in the prediction of mortality of ICU patients (the full challenge description can be found [here](https://physionet.org/content/challenge-2012/1.0.0/)), I implemented a causal discovery pc algorithm using the causal-learn package. This is an easy package to get started with causal discovery. 
Since we have a panel data, i.e., data depending on time and also on individuals. The dependence on the individuals is assumed to be i.i.d. 

The time range is over $48$ hours and some variables only have a few measurements within the $48$ hours, which makes it hard to use time series models. Therefore I chose to run a PC algorithm with the following conditions: 

1. A time-variable at time i can only cause a time-variable at time $i$ or time $i+1$ or the variable "outcomes".

2. The variables "Age", "Weight", "Height" and "Gender" cannot be caused by any time-variables.

3. "Gender" cannot be caused by "Weight", "Height" or "Age".

4. "Height" cannot be caused by "Weight".

5. "outcomes" cannot cause any other variables, it must only have incoming edges.

The time information extracted are the one given above. Each time variables are divided into time intervals, e.g., we aggregate the data with a time interval of, for example, 4 hours. For example, the variable for the temperature will be divided into (48/4 = 12) 12 variables Temp0, Temp1,..., Temp11 and each of these variables have 4000 repetitions (the number of patients in set-a).

The two main strong assumptions that are required, which might be unsatisfied for this dataset, are:

1. Causal sufficiency

2. linear relations between the variables

The first assumption can be relaxed if we use more sophisticated algorithms, 

The second assumption can also be relaxed, however, I chose to assume it here, since most of the patients have different observed variables. This is then possible to predict the outcomes if we don't have all the causal parents. One can then take the observed variables, in some sense, to reconstruct the causal parents and predict the outcome. The coefficients of the path from a variable used for the regression to the outcomes can be multiplied together to get the intervention of this variable on the outcome.

### Class description
I organized my code in three classes:

1. DataCleaning

2. CausalGraph

3. Estimator

The first one cleans the data and transform it to use it for causal discovery and inference. The second class provides the function to do causal discovery and infer the relation between the variables and the last one puts everything together and provide the estimator. 

More precisely, the class DataCleaning perform the following: 
1. Transform the data set from a long format to a wide format.

2. Aggregate the data by averaging out every datapoints within a specified time interval, e.g., 8 hours.

3. We then impute the missing values. We can specified the method between forwardfill and knn, default is forwardfill which is convenient for time series data. For every patients, we impute the missing values of the variables that have been observed but have not been enough measured so that every time interval has a value. 

4. We remove the the outliers using a machine learning algorithm, IsolationForest, and impute them again.

5. Finally, we create two datasets with all patients and all variables, with one column for each variables and one row for each patients. The time series variables have been splited in for each time range, which increase the number of variables. The two datasets are 'self.dataset' and 'self.dataset_imputed'. Both are transformed using 1. - 5. However, for 'self.dataset_imputed' we impute once more, so that every patients have every variables for all times. In the 'self.dataset', some patients(almost all) have some variables completely missing but if they have some values for some times then they have values for all times due to point 3.

6. The method 'cleanDataset' run each of these method one after the other. It just take the raw dataset, e.g., set-a and output the cleaned dataset described above.

The class CausalGraph executes the PC algorithm with the backgroundknowledge conditions as follows:

1. It sets the backgroundknowledge conditions between the variables as described above.

2. It compute the causal graph using the pc algorithm with the background knowledge. In a reasonable time it can get the causal graph for up to about 400 variables. It takes about 4 hours. 

3. We remove the eventual cycle that the graph has created. I am not sure whether it is a mistake from the causal-learn package but it may have cycles.

4. In the method 'setCoefMatrix', we compute all linear coefficients of all edges in the graph.

5. With the method 'setRootNodes', we compute the mean values and store the mean values of the root nodes. We will use them later.

6. Given the data for one patient, the method 'setPredictors' compute all the regressor of this patient that will be used for the prediction. It takes all the causal parents of the variable 'outcomes' and stop if all these variables are observed. If not, it takes the parents of the unobserved variables and, like this, recursively, until all variables are either observed or a root node. the coefficients of all causal path from the observed variables, that will be included in the regression, to the outcomes are calculated by multiplying out all the coefficients of all edges in the path together.

6. ' What could have been done is to take not only the causal parents but also the child of the unobserved parents. They might contain useful information for prediction but not for causal information.

7. Finally, predict the 'outcomes' variable using the variables and coefficients computed in the last method. 


The class estimator regroups the two classes DataCleaning and CausalGraph to get the prediction for the outcomes. 

1. The method 'fit' executes all the step from the class CausalGraph described above.

2. After having predicted the outcomes using the test set 'set-c', we get an array of predictions. Since we are using linear regression, the predictions are continuous. Therefore we have to map the predictions to binary values. The method 'setThreshold', computes the best threshold using only the training dataset 'set-a' and then classifies the predicted outcomes if they are above this threshhold it outputs the value 1 and 0 if it is below the threshold.

3. The method 'getScore' gets the score of the event 1, that is, the minimum between the sensitivity and the positive predictivity.

4. The last method computes all the predictions vs residuals and Q-Q plots for all variables in the causal graph. The plots can be found on [GitHub](https://github.com/SamuelJoray/Predicting-Mortality-of-ICU-Patients/plots_16H).

### Result 
The algorithm for the time_intervals: 4H,8H,12H and 16H have been run.

The runs with 4H, 8H and 12H performs better than the one for the time interval equal to 16H. 

Since there are more than 100 variables for all of these graphs and even more than 400 variables for the causal graph with time interval equal to 4H. However, we can make some remark. The following table gives a summary of the main results. In the column 'Variable', every causal parents of the outcomes are listed, in the column 'Coef.' there are the respective coefficients of the variables and in the column 'Importance' it is the importance of all coefficients, that is, the standardized coefficients, which are $\alpha_{stand.} = \alpha*\frac{sd(predictor)}{sd(outcomes)}$.




| Time interval: 4H | Score: 0.44|               | Time interval: 8H |Score: 0.44 |               | Time interval: 12H|Score: 0.44|               | Time interval: 16H|Score: 0.41|               |
|-------------------|------------|---------------|-------------------|------------|---------------|-------------------|-----------|---------------|-------------------|-----------|---------------|
| **Variable**      | **Coef.**  | **Importance**| **Variable**      | **Coef.**  | **Importance**| **Variable**      | **Coef.** | **Importance**| **Variable**      | **Coef.** | **Importance**|
| Lactate11         | 0.037      | 0.129         | Lactate5          | 0.032      | 0.114         | Lactate3          | 0.033     | 0.118         | Lactate2          | 0.030     | 0.106         |
| Glucose11         | 0.0005     | 0.058         | Glucose5          | 0.0005     | 0.058         | Glucose3          | 0.001     | 0.061         |                   |           |               |
| GCS11             | -0.029     | -0.295        | GCS5              | -0.028     | -0.289        | GCS3              | -0.030    | -0.302        | GCS2              | -0.028    | -0.28         |
| FiO23             | 0.039      | 0.017         | FiO21             | 0.118      | 0.049         |                   |           |               |                   |           |               |
|                   |            |               | WBC5              | 0.003      | 0.060         | WBC3              | 0.003     | 0.059         | WBC2              | 0.002     | 0.039         |
| PaO20             | -0.0002    | -0.070        |                   |            |               |                   |           |               |                   |           |               |
| Urine0            | -0.00007   | -0.058        | Urine5            | -0.0001    | -0.061        |                   |           |               |                   |           |               |
| Urine11           | -0.0001    | -0.052        |                   |            |               |                   |           |               |                   |           |               |
| Weight            | -0.001     | -0.066        | Weight            | -0.001     | -0.067        | Weight            | -0.001    | -0.063        | Weight            | -0.001    | -0.066        |
|                   |            |               |                   |            |               | RespRate0         | 0.008     | 0.066         | RespRate0         | 0.008     | 0.064         |
|                   |            |               |                   |            |               |                   |           |               | BUN2              | 0.003     | 0.184         |
|                   |            |               |                   |            |               |                   |           |               | Albumin0          | -0.041    | -0.058        |


For the time dependent variables, the one that often causes the outcome is the last one, such as, 'Lactate11' whent time interval equals to 4H, 'Lactate5' for 8H, 'Lactate3' for 12H or 'Lactate2' for 16H. All these variables are the ones for the last time interval for the variable 'Lactate'. The same remark holds for the variables 'Glucose', 'WBC', 'GCS'.

Out of the 40 variables there are only a few that often showed up in the regression, namely, 'Glucose', 'WBC', 'GCS', 'Lactate', 'Weight' which indicates a consistency in the algorithm. THe coefficients of these variables is also very close to each other.

According to these results, the coefficient that has the most influence on the outcome is the [Glasgow Coma Score (GCS)](https://en.wikipedia.org/wiki/Glasgow_Coma_Scale) at the end of the stay of the patient, which is used to measure a person's level of consciousness after a brain injury. The score is large for healthy people and low for injured ones. The coefficient is always negative which indicates that when the score is high the outcomes is small, thus there are less risk for the patient.



### Running the code

To simplify the user, everything that takes some time to be run has been saved in a file. But of course, one can re-run everything. Prior to run the code the project has to be downloaded on [GitHub](https://github.com/SamuelJoray/Predicting-Mortality-of-ICU-Patients). The code that is shown in the following can also be found on GitHub after the class Estimator. 

The datasets can be uploaded as follows:

In [ ]:


from Estimator import Estimator
time_interval = 8
directory = #To be filled, the path for the directory where the project is loaded
est = Estimator(time_interval=time_interval, directory=directory)
est.setDirectories()
est.openFiles()

To predict for a patient:

In [ ]:
new_patient= pd.DataFrame()
est.predict(new_patient)

To get the score: 

In [ ]:
est.getScore()

To get the coefficients:

In [ ]:
parents = est.graph.getParents("outcomes")
print(est.graph.coef_matrix[parents].loc["outcomes"])

To get the importance of the coefficients:

In [ ]:
est = Estimator(time_interval=time_interval)
est.setDirectories()

est.openFiles()

parents = est.graph.getParents("outcomes")
est.graph.setCoefMatrix()

print(est.graph.coef_importance[parents].loc["outcomes"])